In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import time
import json
import os 

In [2]:
# Load dataset
data = pd.read_csv('JSE_clean_truncated.csv')

# Define horizons and input windows
horizons = [1, 2, 5, 10, 30]
input_windows = [30, 60, 120]

# Device configuration (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TCN(nn.Module):
    def __init__(self, input_dim, output_dim, num_channels, kernel_size=2, dropout=0.2):
        super(TCN, self).__init__()
        self.num_channels = num_channels
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        layers = []
        for i in range(len(num_channels) - 1):
            layers.append(nn.Conv1d(in_channels=num_channels[i], out_channels=num_channels[i+1], kernel_size=kernel_size, dilation=2**i))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        
        self.tcn = nn.Sequential(*layers)
        self.output_layer = nn.Linear(num_channels[-1], output_dim)
    
    def forward(self, x):
        x = self.tcn(x)
        x = x[:, :, -1]  # Get the last output
        return self.output_layer(x)

In [3]:
# Function to create sliding window input-output pairs
def create_sequences(data, window_size, horizon):
    X = []
    y = []
    for i in range(len(data) - window_size - horizon):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size:i+window_size+horizon])
    return np.array(X), np.array(y)

In [4]:
# Training function
def train_model(model, train_loader, optimizer, num_epochs=100):
    model.train()
    criterion = nn.MSELoss()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

In [5]:
#  Evaluation function
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test).cpu().numpy()
        y_test = y_test.cpu().numpy()
        
        mae = mean_absolute_error(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        return float(mae), float(mape), float(rmse)

In [6]:
# Grid search function
def grid_search(company_idx, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, horizon):
    best_rmse = float('inf')
    best_params = {}
    
    # Hyperparameters to search
    kernel_sizes = [2, 3]
    num_channels_list = [[1, 16, 32], [1, 32, 64]]
    dilation_factors = [1, 2]

    for kernel_size in kernel_sizes:
        for num_channels in num_channels_list:
            for dilation in dilation_factors:
                # Initialize the model
                model = TCN(input_dim=1, output_dim=horizon, num_channels=num_channels, 
                            kernel_size=kernel_size).to(device)
                
                # Define the optimizer
                optimizer = optim.Adam(model.parameters(), lr=0.001)

                # Start time tracking
                start_time = time.time()

                # Train the model
                train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
                train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
                train_model(model, train_loader, optimizer)

                # End time tracking
                end_time = time.time()
                training_time = end_time - start_time
                
                # Evaluate the model
                mae, mape, rmse = evaluate_model(model, X_test_tensor, y_test_tensor)
                
                # Track the best performance
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_params = {
                        'kernel_size': kernel_size,
                        'num_channels': num_channels,
                        'dilation': dilation,
                        'mae': mae,
                        'mape': mape,
                        'rmse': rmse,
                        'training_time': training_time
                    }
    
    return best_params

In [7]:
# Save results to a JSON file (append, don't overwrite)
def save_results_to_json(result, file_name):
    # Check if the file already exists
    if os.path.exists(file_name):
        # If it exists, read the existing content
        with open(file_name, "r") as f:
            existing_data = json.load(f)
    else:
        # If it doesn't exist, create an empty list
        existing_data = []

    # Append the new result to the existing data
    existing_data.append(result)

    # Write back the updated data to the JSON file
    with open(file_name, "w") as f:
        json.dump(existing_data, f, indent=4)

In [ ]:
for company_idx in range(data.shape[1]):
    print(f"\nTraining for company {company_idx+1}")
    
    # Extract data for the current company
    company_data = data.iloc[:, company_idx].values

    # Normalize the company data
    company_data = (company_data - np.mean(company_data)) / np.std(company_data)

    for horizon in horizons:
        for input_window in input_windows:
            print(f"\n--- Horizon: {horizon}, Input Window: {input_window} ---")

            # Create sequences for the current horizon and input window
            X, y = create_sequences(company_data, input_window, horizon)

            # Split the data into train and test sets (e.g., 80-20 split)
            split_idx = int(0.8 * len(X))
            X_train, y_train = X[:split_idx], y[:split_idx]
            X_test, y_test = X[split_idx:], y[split_idx:]

            # Convert data to PyTorch tensors
            X_train_tensor = torch.Tensor(X_train).unsqueeze(1).to(device)  # Adding a channel dimension
            y_train_tensor = torch.Tensor(y_train).to(device)
            X_test_tensor = torch.Tensor(X_test).unsqueeze(1).to(device)
            y_test_tensor = torch.Tensor(y_test).to(device)

            # Perform grid search
            best_params = grid_search(company_idx, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, horizon)

            # Store and print results
            result = {
                "company": company_idx + 1,
                "horizon": horizon,
                "input_window": input_window,
                "run_time": best_params["training_time"],
                "mae": best_params["mae"],
                "mape": best_params["mape"],
                "rmse": best_params["rmse"],
                "optimal_hyperparameters": {
                    "kernel_size": best_params["kernel_size"],
                    "num_channels": best_params["num_channels"],
                    "dilation": best_params["dilation"]
                }
            }
            
            # Print best params
            print(f"\nBest Params for Company {company_idx+1}:")
            print(json.dumps(result, indent=4))

            # Save results to a JSON file (append instead of overwrite)
            file_name = f"company_{company_idx+1}.1_results.json"
            save_results_to_json(result, file_name)


Training for company 1

--- Horizon: 1, Input Window: 30 ---

Best Params for Company 1:
{
    "company": 1,
    "horizon": 1,
    "input_window": 30,
    "run_time": 30.17138385772705,
    "mae": 0.028898876160383224,
    "mape": 4.514417424798012,
    "rmse": 0.04118187353014946,
    "optimal_hyperparameters": {
        "kernel_size": 3,
        "num_channels": [
            1,
            16,
            32
        ],
        "dilation": 2
    }
}

--- Horizon: 1, Input Window: 60 ---
